In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
from torch.utils.data import DataLoader,Dataset
from tqdm import tqdm

In [3]:
covid_train = pd.read_csv('data/covid/covid.train.csv').values
covid_test = pd.read_csv('data/covid/covid.test.csv').values
covid_train.shape

(2699, 118)

## Data Preprocessing

In [4]:
def train_valid_split(train_data,ratio,seed=42):
    np.random.seed(seed)
    rand_index = np.random.permutation(len(train_data))
    vaild_index = rand_index[:int(len(train_data)*ratio)]
    train_index = rand_index[int(len(train_data)*ratio):]
    return train_data[train_index],train_data[vaild_index]
    
def select_feat(train_data, valid_data, test_data,):
    y_train,y_vaild = train_data[:,-1],valid_data[:,-1]
    x_train,x_vaild,x_test = train_data[:,:-1],valid_data[:,:-1],test_data
    return x_train,x_vaild,x_test,y_train,y_vaild
train_data, valid_data = train_valid_split(covid_train,.2)
x_train,x_vaild,x_test,y_train,y_vaild = select_feat(train_data, valid_data,covid_test)
x_train.shape,x_vaild.shape,x_test.shape

((2160, 117), (539, 117), (1078, 117))

In [5]:
class covid_dataset(Dataset):
    
    def __init__(self,x,y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)
    
    def __getitem__(self,idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx],self.y[idx]
    
    def __len__(self):
        return len(self.x)

In [7]:
batch = 50
train_loader = DataLoader(covid_dataset(x_train,y_train),batch_size=batch)
vaild_loader = DataLoader(covid_dataset(x_vaild,y_vaild),batch_size=batch)
test_loader = DataLoader(covid_dataset(x_test),batch_size=batch)
for x,y in train_loader:
    print(f'x:{x.shape}\ny:{y.shape}')
    break

x:torch.Size([50, 117])
y:torch.Size([50])


## Network Model

In [8]:
class net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(117,50),
            torch.nn.ReLU(),
            torch.nn.Linear(50,8),
            torch.nn.ReLU(),
            torch.nn.Linear(8,1),
        )
    
    def forward(self,x):
        x = self.layers(x)
        x = x.squeeze()
        return x
model = net().to('cuda')

In [11]:
loss_f = torch.nn.MSELoss()

optimizer = torch.optim.SGD(model.parameters(),lr=1e-6)
for x,y in train_loader:
    x,y = x.to('cuda'),y.to('cuda')

    pred = model(x)
    loss = loss_f(pred,y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    #train_bar.set_description(f'ROUND:[{size*len(x)}]')
    print(f'LOSS:{loss.item()}')

LOSS:55.868568420410156
LOSS:47.04803466796875
LOSS:40.3697395324707
LOSS:48.98192596435547
LOSS:52.76850509643555
LOSS:40.193809509277344
LOSS:28.20741844177246
LOSS:40.78955841064453
LOSS:54.890926361083984
LOSS:29.272499084472656
LOSS:37.10465621948242
LOSS:25.16103172302246
LOSS:39.4931526184082
LOSS:60.313270568847656
LOSS:24.22102928161621
LOSS:37.65277099609375
LOSS:34.645912170410156
LOSS:31.334867477416992
LOSS:45.25757598876953
LOSS:33.172908782958984
LOSS:40.95221710205078
LOSS:43.10895538330078
LOSS:42.103797912597656
LOSS:46.00162124633789
LOSS:52.064212799072266
LOSS:25.23878288269043
LOSS:35.502323150634766
LOSS:52.84831237792969
LOSS:34.1417121887207
LOSS:38.546302795410156
LOSS:30.14503288269043
LOSS:42.33054733276367
LOSS:67.22917938232422
LOSS:37.493797302246094
LOSS:37.64086151123047
LOSS:32.84696578979492
LOSS:37.30588912963867
LOSS:40.96281051635742
LOSS:38.973262786865234
LOSS:51.37166976928711
LOSS:31.410594940185547
LOSS:32.187416076660156
LOSS:57.2787780761718

## Train & Test

In [109]:
train_bar = tqdm(train_loader,leave=True)


  0%|                                                                                           | 0/44 [00:00<?, ?it/s]

In [15]:
loss_f = torch.nn.MSELoss(reduction='mean')

optimizer = torch.optim.Adam(model.parameters(),lr=1e-5)

def train(train_loader,vaild_loader,model,loss_f,optimizer):
    size = len(train_loader.dataset)
    model.train()
    
    train_bar = tqdm(train_loader,leave=True)
    
    for x,y in train_bar:
        x,y = x.to('cuda'),y.to('cuda')
        optimizer.zero_grad()
        pred = model(x)
        loss = loss_f(pred,y)
        loss.backward()
        optimizer.step()
        
        train_bar.set_description(f'ROUND:[{size*len(x)}]')
        print(f'LOSS:{loss.item()}')
     
    vaild_loss = []                              
    for x_,y_ in vaild_loader:
        x_,y_ = x_.to('cuda'),y_.to('cuda')
        with torch.no_grad():
            pred_ = model(x_)
            vaild_loss.append(loss_f(pred_,y_).item())
    print(f'VAILD_LOSS:{sum(vaild_loss)/len(vaild_loss)}\n--------------------')
    
    return sum(vaild_loss)/len(vaild_loss)
                          

'''def test(test_loader,model,loss_f):
    for x_,y_ in vaild_loader:
        x_,y_ = x_.to('cuda'),y_.to('cuda')
        with torch.no_grad():
            pred_ = model(x_)
            vaild_loss.append(loss_f(pred_,y_).item())
        print(f'VAILD_LOSS:{sum(vaild_loss)/len(vaild_loss)}\n--------------------')'''
    

"def test(test_loader,model,loss_f):\n    for x_,y_ in vaild_loader:\n        x_,y_ = x_.to('cuda'),y_.to('cuda')\n        with torch.no_grad():\n            pred_ = model(x_)\n            vaild_loss.append(loss_f(pred_,y_).item())\n        print(f'VAILD_LOSS:{sum(vaild_loss)/len(vaild_loss)}\n--------------------')"

## START!

In [16]:
times = 0
loss_1 = float('inf')
loss_2 = 0
while (loss_1-loss_2)**2 > 1e-4:
    times+=1
    print(f'TIMES:{times}**************************')
    if times%2 == 0:
        loss_1 = train(train_loader,vaild_loader,model,loss_f,optimizer)
    else:
        loss_2 = train(train_loader,vaild_loader,model,loss_f,optimizer)


TIMES:1**************************


ROUND:[108000]:  41%|██████████████████████████▌                                      | 18/44 [00:00<00:00, 171.11it/s]

LOSS:16.811059951782227
LOSS:11.919508934020996
LOSS:10.23923397064209
LOSS:12.38637638092041
LOSS:13.701069831848145
LOSS:12.370148658752441
LOSS:8.351737022399902
LOSS:10.161428451538086
LOSS:13.823870658874512
LOSS:9.026384353637695
LOSS:9.619860649108887
LOSS:7.88649320602417
LOSS:7.580702781677246
LOSS:13.69171142578125
LOSS:4.6244659423828125
LOSS:7.80018949508667
LOSS:4.381358623504639
LOSS:7.674091815948486
LOSS:9.573267936706543
LOSS:7.371330261230469
LOSS:6.592986106872559
LOSS:8.302163124084473
LOSS:9.436378479003906
LOSS:5.167487144470215
LOSS:7.784244060516357
LOSS:5.591834545135498
LOSS:8.350367546081543
LOSS:9.704893112182617
LOSS:7.159940242767334
LOSS:9.57483196258545


ROUND:[108000]:  41%|██████████████████████████▌                                      | 18/44 [00:00<00:00, 171.11it/s]

LOSS:6.272249698638916
LOSS:7.003725528717041
LOSS:8.143576622009277


...

LOSS:2.9537172317504883
LOSS:3.973148822784424
LOSS:4.82951545715332
LOSS:2.895742654800415


ROUND:[108000]:  66%|██████████████████████████████████████████▊                      | 29/44 [00:00<00:00, 282.31it/s]

LOSS:3.32558274269104
LOSS:3.735626697540283
LOSS:4.717066764831543
LOSS:3.288201332092285
LOSS:4.4901862144470215


ROUND:[21600]: 100%|██████████████████████████████████████████████████████████████████| 44/44 [00:00<00:00, 282.81it/s]


LOSS:2.5220797061920166
LOSS:3.407613754272461
LOSS:3.728771924972534
LOSS:5.732280254364014
LOSS:2.4993674755096436
VAILD_LOSS:4.345583937384865
--------------------
